In [35]:
import os
from openai import OpenAI
from dotenv import load_dotenv

os.chdir('/Users/reejungkim/Documents/Git/working-in-progress')
load_dotenv()


client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["huggingface_read"],
)

completion = client.chat.completions.create(
    model="openai/gpt-oss-20b:novita",
    messages=[
        {
            "role": "user",
            "content": "랭스미스에 대해 쉽게 설명해줘"
        }
    ],
    #response_format={"type": "json_object"}
)

print(completion.choices[0].message.content)  

{"response":"**랭스미스(LangSmith)는 LLM(대형 언어 모델) 개발자와 데이터 과학자들을 위해 만들어진 “모니터링·디버깅·분석” 플랫폼입니다.**\n\n### 1️⃣ 핵심 아이디어\n- **LLM 애플리케이션을 눈으로 확인**할 수 있도록, “챗봇·대화형 애플리케이션”에 발생하는 모든 이벤트(프롬프트, 모델 응답, 메타데이터, 비용 등)를 기록해둡니다.\n- 기록된 데이터를 시각화(그래프, 테이블)하면 “어디서, 왜, 얼마만큼” 잘못됐는지 한눈에 파악할 수 있습니다.\n\n### 2️⃣ 주요 기능\n| 기능 | 예시 | 효과 |\n|------|------|------|\n| **로깅** | 발화한 프롬프트와 모델의 응답을 저장 | 나중에 같은 상황을 재현 가능 |\n| **트레이싱** | 프롬프트 → 토큰화 → 모델 → 응답 → 후처리까지 단계별 추적 | 어느 단계에서 지연이 발생했는지 식별 |\n| **시각화** | 응답 시간, 토큰 수, 비용 등의 KPI를 차트로 | 성능 흐름을 한 눈에 이해 |\n| **디버깅** | 특정 사용자가 받은 ‘잘못된’ 답변을 재생성 → 원인 분석 | hallucination(허위정보) 같은 이슈를 빠르게 해결 |\n| **A/B 테스트** | 같은 상황에서 서로 다른 프롬프트/모델를 비교 | 최적의 설정을 찾는 실험이 쉬워짐 |\n| **메트릭 수집** | 토큰 사용량, 시간, 비용, 에러율 등 | 운영 비용 예측 및 최적화 |\n| **통합** | LangChain·LangSmith·OpenAI·Azure·AWS 등 연동 | 기존에 사용하던 프레임워크와 바로 연결 가능 |\n\n### 3️⃣ 사용법 (간단 예시)\n```python\nfrom langsmith import Client\nclient = Client(api_key=\"YOUR_API_KEY\")\n\n# 1. 프롬프트 카드를 만들어 기록\nclient.create_run(\n    name=\"음식 추천 챗봇 테스트\",\n    inp

In [50]:
generated_text = completion.choices[0].message.content


In [52]:
import json

def parse_response(response_str):
    data = json.loads(response_str)
    return data["response"].replace("\\n", "\n")

# 사용
clean_text = parse_response(generated_text)
print(clean_text)


**랭스미스(LangSmith)는 LLM(대형 언어 모델) 개발자와 데이터 과학자들을 위해 만들어진 “모니터링·디버깅·분석” 플랫폼입니다.**

### 1️⃣ 핵심 아이디어
- **LLM 애플리케이션을 눈으로 확인**할 수 있도록, “챗봇·대화형 애플리케이션”에 발생하는 모든 이벤트(프롬프트, 모델 응답, 메타데이터, 비용 등)를 기록해둡니다.
- 기록된 데이터를 시각화(그래프, 테이블)하면 “어디서, 왜, 얼마만큼” 잘못됐는지 한눈에 파악할 수 있습니다.

### 2️⃣ 주요 기능
| 기능 | 예시 | 효과 |
|------|------|------|
| **로깅** | 발화한 프롬프트와 모델의 응답을 저장 | 나중에 같은 상황을 재현 가능 |
| **트레이싱** | 프롬프트 → 토큰화 → 모델 → 응답 → 후처리까지 단계별 추적 | 어느 단계에서 지연이 발생했는지 식별 |
| **시각화** | 응답 시간, 토큰 수, 비용 등의 KPI를 차트로 | 성능 흐름을 한 눈에 이해 |
| **디버깅** | 특정 사용자가 받은 ‘잘못된’ 답변을 재생성 → 원인 분석 | hallucination(허위정보) 같은 이슈를 빠르게 해결 |
| **A/B 테스트** | 같은 상황에서 서로 다른 프롬프트/모델를 비교 | 최적의 설정을 찾는 실험이 쉬워짐 |
| **메트릭 수집** | 토큰 사용량, 시간, 비용, 에러율 등 | 운영 비용 예측 및 최적화 |
| **통합** | LangChain·LangSmith·OpenAI·Azure·AWS 등 연동 | 기존에 사용하던 프레임워크와 바로 연결 가능 |

### 3️⃣ 사용법 (간단 예시)
```python
from langsmith import Client
client = Client(api_key="YOUR_API_KEY")

# 1. 프롬프트 카드를 만들어 기록
client.create_run(
    name="음식 추천 챗봇 테스트",
    inputs={"question": "오늘 점심에 뭐 먹지?"},
    outp